<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/QM_ML1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit_machine_learning
!pip install TorchConnector
!pip install pandas
!pip install numpy

import pandas as pd
import numpy as np
from qiskit_aer import Aer # Correct import for Aer
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit.algorithms import ADAM, optimizers

# Generating synthetic financial data
np.random.seed(0)
dates = pd.date_range(start='2023-01-01', periods=100, freq='D')
prices = np.random.normal(loc=100, scale=10, size=(100,))

# Preparing the data
data = pd.DataFrame(data={'Date': dates, 'Price': prices})
data['Returns'] = data['Price'].pct_change()
data.dropna(inplace=True)
X = data['Returns'].values.reshape(-1, 1)
Y = (data['Returns'].values > 0).astype(int)

# Splitting the data into training and testing sets
split_ratio = 0.8
split = int(split_ratio * len(X))
X_train, X_test = X[:split], X[split:]
Y_train, Y_test = Y[:split], Y[split:]

# Setting up the quantum instance
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

# Building the Quantum Circuit
feature_dim = 1
feature_map = QuantumCircuit(feature_dim)
params = ParameterVector('params', length=feature_dim)
feature_map.ry(params[0], 0)

ansatz = QuantumCircuit(feature_dim)
params_ansatz = ParameterVector('params_ansatz', length=3)  # Three parameters for a single qubit
ansatz.rx(params_ansatz[0], 0)
ansatz.ry(params_ansatz[1], 0)
ansatz.rz(params_ansatz[2], 0)

# Combining feature map and ansatz
circuit = feature_map.compose(ansatz)
qnn = CircuitQNN(circuit, feature_map.params, ansatz.params, sparse=True,
                 output_shape=2, quantum_instance=quantum_instance)

# Creating and training the classifier
classifier = NeuralNetworkClassifier(qnn, optimizer=ADAM(learning_rate=0.1))
classifier.fit(X_train, Y_train)

# Making predictions
predicted = classifier.predict(X_test)
accuracy = np.sum(predicted == Y_test) / len(Y_test)
print(f'Prediction accuracy: {accuracy}')

# Generate buy (1) or sell (0) signals based on predictions
signals = ['Buy' if pred == 1 else 'Sell' for pred in predicted]
print(signals)

ERROR: Could not find a version that satisfies the requirement TorchConnector (from versions: none)
ERROR: No matching distribution found for TorchConnector


ImportError: cannot import name 'CircuitQNN' from 'qiskit_machine_learning.neural_networks' (/usr/local/lib/python3.11/dist-packages/qiskit_machine_learning/neural_networks/__init__.py)